In [ ]:
!pip install llama-index-llms-groq
!pip install llama-index
!pip install pdfplumber
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.llms.groq import Groq
import pdfplumber
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np


In [ ]:
# Create an API key at the Groq console (https://console.groq.com/keys)
# Then set it to the environment variable GROQ_API_KEY or pass it directly

api_key = "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"  # Replace with your actual API key
llm = Groq(model="llama3-70b-8192", api_key=api_key)


In [ ]:
def extract_text_with_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Example usage
pdf_path = "/content/agenda.pdf"  # Replace with the path to your PDF file
extracted_text = extract_text_with_pdfplumber(pdf_path)
print(extracted_text)


Agenda
Global IndiaAI Summit 2024 3-4 July 2024, New Delhi
Day 1 - Wednesday, July 3
Start Session Description
10:00 - 11:00
Opening Ceremony (Auditorium 1)
11:00 – 13:30
GPAI Session (@ Summit Room) : Executive Council Meeting
(By Invitation only)
11:30 - 13:00 Side Event 1 (@ The session is about learning and sharing knowledge on Large
Auditorium 1): Language Models (LLMs) and Large Multimodal Models (LMMs),
with the aim to understand the unique linguistic and cultural
IndiaAI: Large diversity inherent to India, examining how LLMs can effectively
Language Model address challenges associated with multilingualism. Furthermore,
the session will cover the ethical considerations and biases linked
to these models, promoting a discussion on responsible AI
practices, including fairness, inclusiveness, misinformation
mitigation, and intellectual property rights within diverse cultural
contexts. Lastly, the session will focus on the collaborative
opportunities among indigenous communities, aca

In [ ]:
# Define a list of questions and their expected answers (true labels)
questions_and_answers = [
    ("What is the name of the summit?", "Global IndiaAI Summit 2024"),
    ("When and where is the Global IndiaAI Summit 2024 taking place?", "3-4 July 2024, New Delhi"),
    ("What is the time and venue for the opening ceremony on Day 1?", "10:00 - 11:00 at Auditorium 1"),
    ("What is the GPAI Session about at 11:00 on Day 1?", "Executive Council Meeting (By Invitation only)"),
]


In [ ]:
# Function to ask questions and get predictions from the model
def ask_questions(text, questions_and_answers):
    true_labels = []
    predicted_labels = []

    for question, true_answer in questions_and_answers:
        true_labels.append(true_answer)
        response = llm.complete(f"Give exact answers in one or two words based on {text}, always give date as first when answer containts date, for the question: {question}")
        predicted_labels.append(response.text)  # Accessing the text attribute directly

    return true_labels, predicted_labels

# Example usage
true_labels, predicted_labels = ask_questions(extracted_text, questions_and_answers)
print("True Labels:")
print(true_labels)
print("Predicted Labels:")
print(predicted_labels)


def evaluate_model(true_labels, predicted_labels):
    correct_count = 0
    total_count = len(true_labels)

    for true_label, predicted_label in zip(true_labels, predicted_labels):
        if true_label in predicted_label or predicted_label in true_label:
            correct_count += 1

    precision = correct_count / total_count
    recall = correct_count / total_count
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

precision, recall, f1 = evaluate_model(true_labels, predicted_labels)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


True Labels:
['Global IndiaAI Summit 2024', '3-4 July 2024, New Delhi', '10:00 - 11:00 at Auditorium 1', 'Executive Council Meeting (By Invitation only)']
Predicted Labels:
['Global IndiaAI Summit 2024', 'July 3-4, 2024, New Delhi', 'July 3, 10:00 - 11:00, Auditorium 1', 'Executive Council Meeting']
Precision: 0.5
Recall: 0.5
F1 Score: 0.5


In [ ]:
# Print each question, true answer, and predicted answer
print("Comparison of True and Predicted Answers:")
for i, (true_label, predicted_label) in enumerate(zip(true_labels, predicted_labels)):
    print(f"Question {i + 1}:")
    print(f"True Answer: {true_label}")
    print(f"Predicted Answer: {predicted_label}")
    if true_label != predicted_label:
        print("--> Score impacted: Check precision and recall for this question.")
    print()


Comparison of True and Predicted Answers:
Question 1:
True Answer: Global IndiaAI Summit 2024
Predicted Answer: Global IndiaAI Summit 2024

Question 2:
True Answer: 3-4 July 2024, New Delhi
Predicted Answer: July 3-4, 2024, New Delhi
--> Score impacted: Check precision and recall for this question.

Question 3:
True Answer: 10:00 - 11:00 at Auditorium 1
Predicted Answer: July 3, 10:00 - 11:00, Auditorium 1
--> Score impacted: Check precision and recall for this question.

Question 4:
True Answer: Executive Council Meeting (By Invitation only)
Predicted Answer: Executive Council Meeting
--> Score impacted: Check precision and recall for this question.



In [ ]:
def evaluate_questions(true_labels, predicted_labels):
    results = []

    for true_label, predicted_label in zip(true_labels, predicted_labels):
        true_set = set(true_label.lower().split())
        predicted_set = set(predicted_label.lower().split())

        # Calculate metrics
        intersection = true_set.intersection(predicted_set)
        precision = len(intersection) / len(predicted_set) if len(predicted_set) > 0 else 0
        recall = len(intersection) / len(true_set) if len(true_set) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        results.append({
            "True Label": true_label,
            "Predicted Label": predicted_label,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        })

    return results

# Evaluate questions
evaluation_results = evaluate_questions(true_labels, predicted_labels)

# Display results for each question
for i, result in enumerate(evaluation_results):
    print(f"Question {i + 1}:")
    print(f"True Label: {result['True Label']}")
    print(f"Predicted Label: {result['Predicted Label']}")
    print(f"Precision: {result['Precision']:.2f}")
    print(f"Recall: {result['Recall']:.2f}")
    print(f"F1 Score: {result['F1 Score']:.2f}")
    print()


Question 1:
True Label: Global IndiaAI Summit 2024
Predicted Label: Global IndiaAI Summit 2024
Precision: 1.00
Recall: 1.00
F1 Score: 1.00

Question 2:
True Label: 3-4 July 2024, New Delhi
Predicted Label: July 3-4, 2024, New Delhi
Precision: 0.80
Recall: 0.80
F1 Score: 0.80

Question 3:
True Label: 10:00 - 11:00 at Auditorium 1
Predicted Label: July 3, 10:00 - 11:00, Auditorium 1
Precision: 0.57
Recall: 0.67
F1 Score: 0.62

Question 4:
True Label: Executive Council Meeting (By Invitation only)
Predicted Label: Executive Council Meeting
Precision: 1.00
Recall: 0.50
F1 Score: 0.67

